# functions

In [1]:
import uuid
from langchain_core.output_parsers import JsonOutputParser

class JsonOrRawParser(JsonOutputParser):
    def invoke(self, input, config=None):
        try:
            tool_calls = super().invoke(input)
            tool_calls = [tool_call | {"id": str(uuid.uuid4()), "type": "tool_call"} for tool_call in tool_calls]
            input.tool_calls = tool_calls
            input.content = ""
            
            return input
            
        except Exception as e:
            return input

In [12]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

class ChatOllamaCustomized(ChatOllama):
    def bind_tools(self, tools):
        rendered_tools = render_text_description(tools)
        system_prompt = f"""\
        You have access to functions. If you decide to invoke any of the function(s),
        you MUST put it in the format of
        json[
          {{{{
            "name": "tool_name",
            "arguments": dictionary of argument name and its value
          }}}},
          ...
        ]
        You SHOULD NOT include any other text in the response if you call a function.
        {rendered_tools}
        """
        prompt = ChatPromptTemplate.from_messages(
            [("system", system_prompt), ("user", "{input}")]
        )
        chain = prompt | llm | JsonOrRawParser()

        return chain.bind()

# setup

In [13]:
base_url="http://host.docker.internal:11434"
model_name = "gemma3:27b"
llm = ChatOllamaCustomized(
    model=model_name,
    base_url=base_url,
)

# tools

In [14]:
from langchain_core.tools import tool


@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together."""
    return a * b


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y

tools = [multiply, add]

# usage

In [15]:
llm = ChatOllamaCustomized(
    model=model_name,
    base_url=base_url,
)

In [16]:
llm_with_tools = llm.bind_tools(tools)

In [18]:
llm_with_tools.invoke("12*7 and 15+8")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gemma3:27b', 'created_at': '2025-05-21T13:28:28.064469726Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2239327612, 'load_duration': 95832384, 'prompt_eval_count': 149, 'prompt_eval_duration': 44729189, 'eval_count': 87, 'eval_duration': 2096660456, 'model_name': 'gemma3:27b'}, id='run--a13b9783-dc90-4089-b92c-baf004a422d9-0', tool_calls=[{'name': 'multiply', 'arguments': {'a': 12, 'b': 7}, 'id': '55bb3b42-d775-464d-a6db-2efe48d7f7c5', 'type': 'tool_call'}, {'name': 'add', 'arguments': {'x': 15, 'y': 8}, 'id': '6779fc31-fc9f-47bd-a1ee-4aa1366c5c5d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 149, 'output_tokens': 87, 'total_tokens': 236})